### Предсказание погоды - лабораторная работа 

## ИУ5 ОАД

В качестве лабораторной работы вам предлагается поработать над предсказанием погоды. Файл с данными вы найдете в соответствующей директории. Вам будет доступен датасет weather.csv, ПЕРВЫЕ 75% (shuffle = False) которого нужно взять для обучения, последние 25% - для тестирования.

Требуется построить модель которая будутет предсказывать целевую переменную <b>RainTomorrow</b> с помощью логистической регрессии 

Не забывайте о том, что работа с признаками играет очень большую роль в построении хорошей модели.

Краткое описание данных:

    Date - Дата наблюдений
    Location - Название локации, в которой расположена метеорологическая станция
    MinTemp - Минимальная температура в градусах цельсия
    MaxTemp - Максимальная температура в градусах цельсия
    Rainfall - Количество осадков, зафиксированных за день в мм
    Evaporation - Так называемое "pan evaporation" класса А (мм) за 24 часа до 9 утра
    Sunshine - Число солнечных часов за день
    WindGustDir - направление самого сильного порыва ветра за последние 24 часа
    WindGustSpeed - скорость (км / ч) самого сильного порыва ветра за последние 24 часа
    WindDir9am - направление ветра в 9 утра

In [1]:
import warnings  # Импортируем модуль для работы с предупреждениями
warnings.filterwarnings("ignore")  # Игнорируем все предупреждения, чтобы они не выводились в консоль

import pandas as pd  # Импортируем pandas для работы с табличными данными
import numpy as np  # Импортируем numpy для работы с массивами и математическими операциями
import seaborn as sns  # Импортируем seaborn для визуализации данных
import time  # Импортируем time для работы с временем
from sklearn.datasets import make_classification  # Импортируем функцию для генерации синтетических данных для классификации
import matplotlib.pyplot as plt  # Импортируем matplotlib для построения графиков

# Активируем режим отображения графиков в Jupyter notebook
%matplotlib notebook

In [2]:
X = pd.read_csv('weather.csv')  # Чтение данных из CSV файла в DataFrame X

In [3]:
y = X.RainTomorrow.replace({'No':0, 'Yes': 1})  # Замена значений в колонке 'RainTomorrow'

In [4]:
del X['RainTomorrow']  # Удаление столбца 'RainTomorrow' из X

### Подготовка данных и исследование моделей на числовых признаках

Посмотрите на данные, которые у нас имеются. 

In [5]:
X  # Отображение данных X

,Unnamed: 0,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,...,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No
1,1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,...,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No
2,2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,...,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No
3,3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,...,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No
4,4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,...,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,145454,2017-06-20,Uluru,3.5,21.8,0.0,NaN,NaN,E,31.0,...,13.0,59.0,27.0,1024.7,1021.2,NaN,NaN,9.4,20.9,No
142189,145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,...,11.0,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No
142190,145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,...,9.0,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No
142191,145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,...,9.0,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No


In [6]:
X.info()  # Информация о DataFrame X

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     142193 non-null  int64  
 1   Date           142193 non-null  object 
 2   Location       142193 non-null  object 
 3   MinTemp        141556 non-null  float64
 4   MaxTemp        141871 non-null  float64
 5   Rainfall       140787 non-null  float64
 6   Evaporation    81350 non-null   float64
 7   Sunshine       74377 non-null   float64
 8   WindGustDir    132863 non-null  object 
 9   WindGustSpeed  132923 non-null  float64
 10  WindDir9am     132180 non-null  object 
 11  WindDir3pm     138415 non-null  object 
 12  WindSpeed9am   140845 non-null  float64
 13  WindSpeed3pm   139563 non-null  float64
 14  Humidity9am    140419 non-null  float64
 15  Humidity3pm    138583 non-null  float64
 16  Pressure9am    128179 non-null  float64
 17  Pressure3pm    128212 non-nul

Посмотрим на пропуски в данных

In [7]:
X.isnull().sum()  # Подсчет пропущенных значений в каждом столбце

Unnamed: 0           0
Date                 0
Location             0
MinTemp            637
MaxTemp            322
Rainfall          1406
Evaporation      60843
Sunshine         67816
WindGustDir       9330
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am         53657
Cloud3pm         57094
Temp9am            904
Temp3pm           2726
RainToday         1406
dtype: int64

Заполните пропуски любым известным способом

In [8]:
# Заполнение пропусков
for column in X.columns:  # Перебираем все столбцы в DataFrame X
    if X[column].dtype == 'object':  # Если тип данных столбца - категориальный (object)
        # Для категориальных признаков заполним пропуски наиболее частым значением
        X[column].fillna(X[column].mode()[0], inplace=True)
        # Используем метод `mode()`, который возвращает наиболее частое значение в столбце.
        # `[0]` выбирает первое значение (в случае, если несколько мод).
        # `inplace=True` выполняет замену в исходном DataFrame (без создания копии).
    else:  # Если столбец не категориальный (т.е. числовой)
        # Для числовых признаков заполним пропуски средним значением
        X[column].fillna(X[column].mean(), inplace=True)
        # Метод `mean()` вычисляет среднее значение по столбцу.
        # Это среднее значение используется для заполнения пропусков.


Проанализируйте данные и возьмите из них только числовые фичи

In [9]:
# Выбор числовых признаков
X_numeric = X.select_dtypes(include=[np.number])  
# С помощью метода `select_dtypes(include=[np.number])` мы выбираем только те столбцы в DataFrame `X`, 
# которые имеют числовые данные (типы данных int64, float64 и т.д.). 
# Таким образом, `X_numeric` будет содержать только числовые признаки.

Разобьём данные на train и test и попробуем обучить на них логистическую регрессию

In [11]:
from sklearn.model_selection import train_test_split  # Импортируем функцию для разделения данных на обучающую и тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(X_numeric, y, test_size=0.25, random_state=10, shuffle=False)
# `train_test_split` используется для разделения данных на обучающую (X_train, y_train) и тестовую (X_test, y_test) выборки.
# - `X_numeric` — это числовые признаки (обработанные данные).
# - `y` — целевая переменная (обработанная категория 'RainTomorrow').
# - `test_size=0.25` означает, что 25% данных будут отведены для тестирования, а 75% для обучения.
# - `random_state=10` фиксирует случайное разделение, чтобы оно было воспроизводимым.
# - `shuffle=False` отключает случайное перемешивание данных, то есть данные будут разделены по порядку.

In [12]:
from sklearn.linear_model import LogisticRegression  # Импортируем модель логистической регрессии

model = LogisticRegression(random_state=0)  # Создаем объект модели логистической регрессии
model.fit(X_train, y_train)  # Обучаем модель на обучающих данных
y_pred = model.predict(X_test)  # Применяем обученную модель для предсказания на тестовых данных

In [13]:
from sklearn.metrics import roc_auc_score  # Импортируем метрику для оценки качества классификации

roc_auc_score(y_test, y_pred)  # Рассчитываем AUC-ROC для предсказанных значений

np.float64(0.6802769973409059)

Попробуйте применить StandardScaler и снова обучить модель. Посмотрите на roc_auc_score и сделайте выводы

In [14]:
from sklearn.preprocessing import StandardScaler  # Импортируем StandardScaler для стандартизации данных


# Стандартизация числовых данных
scaler = StandardScaler()  # Создаем объект StandardScaler
X_train_scaled = scaler.fit_transform(X_train)  # Стандартизируем обучающие данные
X_test_scaled = scaler.transform(X_test)  # Стандартизируем тестовые данные с использованием тех же параметров

# Обучение модели на стандартизированных данных
model = LogisticRegression(random_state=0)  # Создаем объект модели логистической регрессии
model.fit(X_train_scaled, y_train)  # Обучаем модель на стандартизированных обучающих данных
y_pred_scaled = model.predict(X_test_scaled)  # Применяем модель для предсказания на стандартизированных тестовых данных

# Оценка качества модели
roc_auc_scaled = roc_auc_score(y_test, y_pred_scaled)  # Рассчитываем AUC-ROC для предсказанных значений
print("ROC AUC после стандартизации:", roc_auc_scaled)


ROC AUC после стандартизации: 0.6917108126781513


Закодируйте катигориальные фичи и добавьте их в логистическую регрессию (например, направление ветра, месяц и т.д.)

In [15]:
# Преобразование даты для добавления месяца как признака
X['Month'] = pd.to_datetime(X['Date']).dt.month  
# Преобразуем столбец 'Date' в тип данных datetime, затем извлекаем только месяц (значение от 1 до 12) 
# и создаем новый столбец 'Month', который будет содержать номер месяца для каждой строки.

# Выбор категориальных признаков и их кодирование
X_categorical = X[['WindGustDir', 'WindDir9am', 'Location']]  
# Выбираем категориальные признаки для кодирования: 'WindGustDir', 'WindDir9am', 'Location'. 
# Эти признаки, скорее всего, содержат текстовые или категориальные данные.

X_encoded = pd.get_dummies(X_categorical, drop_first=True)  
# Применяем one-hot кодирование к категориальным признакам.
# `get_dummies()` создает новые бинарные столбцы для каждого уникального значения в категориальных признаках.
# Параметр `drop_first=True` исключает первый столбец из кодирования, чтобы избежать мультиколлинеарности.
# Например, если у нас есть категории "N", "S", "E" для 'WindGustDir', то будут созданы два столбца (без первого).

In [16]:
# Объединение числовых и категориальных признаков
X_combined = pd.concat([X_numeric, X_encoded, X['Month']], axis=1)  
# Объединяем числовые признаки (X_numeric), закодированные категориальные признаки (X_encoded) и столбец 'Month' 
# (номер месяца) в один DataFrame X_combined.

# Определение индекса для разделения данных (75% для train, 25% для test)
split_index = int(len(X_combined) * 0.75)  
# Определяем индекс, который будет использоваться для разделения данных на обучающие и тестовые выборки.
# Мы берем 75% данных для обучения и 25% для тестирования.

# Разделение на обучающие и тестовые выборки
X_train_combined, X_test_combined = X_combined[:split_index], X_combined[split_index:]  
y_train, y_test = y[:split_index], y[split_index:]  
# Делаем разделение данных на обучающие (X_train_combined, y_train) и тестовые (X_test_combined, y_test) выборки,
# используя ранее вычисленный индекс.

# Стандартизация данных
scaler = StandardScaler()  
# Создаем объект StandardScaler, который будет использоваться для стандартизации данных.
X_train_combined_scaled = scaler.fit_transform(X_train_combined)  
# Применяем стандартизацию к обучающим данным. fit_transform() вычисляет параметры (среднее и std) и применяет их.
X_test_combined_scaled = scaler.transform(X_test_combined)  
# Применяем стандартизацию к тестовым данным, используя параметры, полученные на обучающих данных.

# Обучение модели на комбинированных данных
model_combined = LogisticRegression(random_state=0)  
# Создаем объект модели логистической регрессии с фиксированным random_state для воспроизводимости.
model_combined.fit(X_train_combined_scaled, y_train)  
# Обучаем модель на стандартизированных обучающих данных (X_train_combined_scaled) и целевой переменной (y_train).
y_pred_combined = model_combined.predict(X_test_combined_scaled)  
# Применяем модель для предсказания значений на тестовых данных (X_test_combined_scaled).


Измерьте качество получившейся модели на тестовых данных и сделайте выводы. Добейтесь roc_auc_score выше 0.7

In [17]:
# Оценка качества модели с использованием AUC-ROC
roc_auc_scaled = roc_auc_score(y_test, y_pred_combined)  
# Рассчитываем AUC-ROC для предсказанных значений (y_pred_combined) по сравнению с реальными метками (y_test).
print("ROC AUC после стандартизации:", roc_auc_scaled)  
# Выводим результат AUC-ROC после стандартизации данных.

ROC AUC после стандартизации: 0.7731427915667667


### Реализация логистической регрессии 
__Логистическая регрессия__

$$p(y|x) = a(x, \theta) = \sigma(\langle x, \theta \rangle) = \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}$$

In [18]:
# Вектор параметров модели (например, коэффициенты для признаков в линейной регрессии или логистической регрессии)
theta = np.array([1, 2, 3])  
# Это массив с тремя элементами, представляющими параметры (или коэффициенты) модели для признаков.

# Матрица признаков X (каждая строка - это один пример, а каждый столбец - это признак)
X = np.array([[ 1,  1, 1],  # Пример 1
              [-1, -2, 1],  # Пример 2
              [-1, -2, 2],  # Пример 3
              [-2, -2, -3]])  # Пример 4
# Это матрица 4x3, где каждый столбец представляет собой отдельный признак (например, температура, влажность и т. д.), 
# а каждая строка — это отдельный пример с этими признаками.

# Целевая переменная y (это метки классов или регрессионные значения)
y = np.array([1, 1, 0, 0])  
# Это вектор с целевыми значениями для 4 примеров. В бинарной классификации 1 может означать "положительный класс", а 0 — "отрицательный".

In [19]:
# Функция для вычисления вероятности с использованием логистической регрессии
def probability(theta, X):
    # Вычисляем скалярное произведение θ и X, затем применяем сигмоид
    result = 1 / (1 + np.exp(-np.dot(X, theta)))  # np.dot(X, theta) вычисляет скалярное произведение X и theta.
    # Функция сигмоида применяется к результату, преобразуя его в значение от 0 до 1.
    return result  # Возвращаем результат, который будет представлять собой вероятность.

# Вычисление вероятности для каждого примера с использованием функции
prob = probability(theta, X)  # Вызываем функцию с входными данными theta и X для получения предсказанных вероятностей.

# Проверки правильности работы функции
assert type(prob) == np.ndarray, 'Возвращается неверный тип'  
# Проверяем, что возвращаемое значение имеет тип numpy.ndarray, как и должно быть для массива вероятностей.

assert prob.shape == (X.shape[0],), 'Неверный размер массива'  
# Проверяем, что размерность возвращаемого массива совпадает с количеством строк в X (по количеству наблюдений).

assert (prob.round(3) == [0.998, 0.119, 0.731, 0.]).all(), 'Функция считается неверно'  
# Проверяем, что округленные вероятности для каждого наблюдения равны ожидаемым значениям.
# Если все значения совпадают, то проверка пройдет успешно.

Функция предсказания метки класса, получает на вход вероятности принадлежности к классу 1 и выдает метки классов $y \in \{0, 1\}$

In [20]:
# Функция для предсказания бинарного класса на основе вероятности
def binary_class_prediction(theta, X, threshold=0.5):
    prob = probability(theta, X)  # Получаем вероятности для каждого примера с использованием функции probability
    # Присваиваем метки 1 или 0 в зависимости от порога
    result = (prob >= threshold).astype(int)  # Если вероятность больше или равна порогу, присваиваем 1, иначе 0
    return result  # Возвращаем массив меток классов (0 или 1)

# Вызываем функцию для предсказания бинарных классов
y_pred = binary_class_prediction(theta, X)

# Проверка правильности работы функции
assert type(y_pred) == np.ndarray, 'Возвращается неверный тип'  
# Проверяем, что возвращаемое значение — это numpy.ndarray.

assert y_pred.shape == (X.shape[0],), 'Неверный размер массива'  
# Проверяем, что размерность возвращаемого массива соответствует количеству примеров в X.

assert min(y_pred) == 0, 'Функция считается неверно'  
# Проверяем, что в массиве y_pred есть хотя бы один элемент, равный 0 (отрицательный класс).

assert max(y_pred) == 1, 'Функция считается неверно'  
# Проверяем, что в массиве y_pred есть хотя бы один элемент, равный 1 (положительный класс).


__Функционал качества логистической регрессии__

Запишем правдободовие выборки для меток класса $y \in \{+1, -1\}$ 

$$Likelihood(a, X^\ell) = \prod_{i = 1}^{\ell} a(x_i,\theta)^{[y_i = +1]} (1 - a(x_i, \theta))^{[y_i = -1]} → \operatorname*{max}_{\theta}$$ 

Прологарифмируем правдоподобие выборки и перейдем к задаче минимизации:

$$Q(a, X^\ell) =     -\sum_{i = 1}^{\ell} 
        [y_i = +1] \log a(x_i, \theta)
        +
        [y_i = -1] \log (1 - a(x_i, \theta)) \to \operatorname*{min}_{\theta}$$ 
        
Подставим $a(x, \theta)$ в функцинал качества:

$$ Q(a, X^\ell) = -\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{\exp(-\langle \theta, x_i \rangle)}{1 + \exp(-\langle \theta, x_i \rangle)}
\right)
=\\
=
-\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{1}{1 + \exp(\langle \theta, x_i \rangle)}
\right)
=\\
=
\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) $$
    

Итоговый оптимизируемый функционал качества (logloss), записанный для меток классов $y \in \{+1, -1\}$ и усредненный по выборке

$$Q(a, X^\ell) = \frac{1}{\ell}\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) \to \operatorname*{min}_{\theta}$$

Реализуем его в функции logloss:

In [21]:
def logloss(theta, X, y):
    # Преобразуем y в {+1, -1}, если она состоит из {1, 0}
    y_transformed = np.where(y == 0, -1, 1)  
    # Преобразуем целевые метки из {0, 1} в {+1, -1}. 
    # Это необходимо для корректного вычисления логарифмических потерь в логистической регрессии.
    # np.where возвращает массив, в котором 0 заменяется на -1, а 1 остается как есть.
    
    # Вычисляем скалярное произведение θ и X, умножаем на метки классов y
    z = y_transformed * np.dot(X, theta)  
    # Мы вычисляем скалярное произведение между матрицей признаков X и вектором параметров θ, 
    # а затем умножаем результат на целевые метки y.
    # Это создаст вектор значений, который используется в логарифмических потерях.

    # Вычисляем логарифм потерь по формуле
    result = np.mean(np.log(1 + np.exp(-z)))  
    # Для каждого элемента z вычисляется логарифм функции потерь.
    # Функция потерь для логистической регрессии: `log(1 + exp(-y * (X.dot(theta))))`.
    # Мы берем среднее значение по всем примерам, чтобы получить окончательное значение потерь.
    
    return result  # Возвращаем итоговый результат логарифмических потерь


In [29]:
assert logloss(theta, X, y).round(3) == 0.861, 'Функция считается неверно'

__Алгоритм оптимизации функционала качества. Стохастический градиентный спуск__

<b>Вход: </b> Выборка $X^\ell$, темп обучения $h$

<b>Выход: </b> оптимальный вектор весов $\theta$

1.  Инициализировать веса $\theta$
2.  Инициализировать оценку функционала качества: $Q(a, X^\ell)$
3.  <b>Повторять</b>: 

    Выбрать случайным образом подвыборку объектов $X^{batch} =\{x_1, \dots,x_n \}$ из $X^{\ell}$
    
    Рассчитать градиент функционала качества: $\nabla Q(X^{batch}, \theta)$
    
    Обновить веса: $\theta := \theta - h\cdot \nabla Q(X^{batch}, \theta)$
       
    <b>Пока</b> значение $Q$ и/или веса $\theta$ не сойдутся   

Реализуем функцию рассчета градиента функционала качества

$$\frac{\partial Q(a, X^{batch}) }{\partial \theta_j}   = \frac{\partial \frac{1}{n}\sum_{i = 1}^{n}
    \log \left(
        1 + \exp(- y_i \langle \theta, x_i \rangle)
    \right)} {\partial \theta_j}  = \frac{1}{n}\sum_{i = 1}^{n}
     \frac {1}{
        1 + \exp(- y_i \langle \theta, x_i \rangle)} \cdot  \exp(- y_i \langle \theta, x_i \rangle) \cdot -y_i x_{ij}$$

Реализуйте рассчет градиента в матричном виде:

In [22]:
def gradient(theta, X, y):
    # Вычисляем предсказания с использованием сигмоиды
    z = np.dot(X, theta)  
    # Вычисляем линейное сочетание признаков (X) и параметров (theta), используя скалярное произведение.

    sigmoid = 1 / (1 + np.exp(-z))  
    # Применяем сигмоидную функцию (логистическую функцию), которая ограничивает результат между 0 и 1.
    # Это дает нам предсказания вероятности для каждого наблюдения.

    # Рассчитываем градиент
    grad = -np.dot(X.T, y * (1 - sigmoid)) / len(y)  
    # Вычисляем градиент функции потерь (по логистической регрессии).
    # - `X.T` - транспонированная матрица признаков.
    # - `y * (1 - sigmoid)` - разница между реальными метками и предсказанными значениями (ошибка).
    # - `np.dot(X.T, y * (1 - sigmoid))` - вычисляем градиент (суммируем ошибки, взвешенные признаками).
    # - Мы делим на количество наблюдений, чтобы получить усредненный градиент.
    
    return grad  # Возвращаем рассчитанный градиент.

# Проверка корректности размера выходного массива
assert gradient(theta, X, y).shape == theta.shape, 'Неверный размер массива'  
# Проверяем, что размер возвращаемого градиента соответствует размеру вектора параметров theta.


Функция обучения уже реализована

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import time  # Импортируем для задержки между итерациями, чтобы обновления графиков были видны

# Функция для обучения модели с использованием градиентного спуска
def fit(X, y, batch_size=10, h=0.05, iters=100, plot=True):

    # Получаем размерности матрицы X (количество примеров и количество признаков)
    size, dim = X.shape

    # Случайная инициализация вектора параметров (theta) с размерами, равными количеству признаков
    theta = np.random.uniform(size=dim)

    # Список для хранения значений ошибки на каждой итерации
    errors = []

    # История значений параметров theta на каждой итерации
    theta_history = theta
    # Генерация цветов для визуализации параметров (будет использоваться для отображения графиков)
    colors = [plt.get_cmap('gist_rainbow')(i) for i in np.linspace(0,1,dim)]

    # Если необходимо построить графики
    if plot:
        # Создаем фигуру и три подграфика
        fig = plt.figure(figsize=(15, 10))
        ax1 = fig.add_subplot(221)  # Первый подграфик (для theta)
        ax2 = fig.add_subplot(222)  # Второй подграфик (для логарифмической потери)
        ax3 = fig.add_subplot(212)  # Третий подграфик (для истории изменения theta)
        fig.suptitle('Gradient descent')  # Заголовок всей фигуры

    # Основной цикл градиентного спуска
    for _ in range(iters):

        # Выбираем случайную подвыборку для батча
        batch = np.random.choice(size, batch_size, replace=False)  # Индексы случайных примеров
        X_batch = X[batch]  # Выбираем соответствующие данные
        y_batch = y[batch]  # Выбираем соответствующие метки

        # Вычисляем градиент функции потерь для текущего батча
        grad = gradient(theta, X_batch, y_batch)  # Функция gradient должна быть определена ранее

        # Проверка правильности вычисления градиента
        assert type(grad) == np.ndarray, 'неверный тип'  # Проверяем, что градиент - это numpy-массив
        assert len(grad.shape) == 1, 'Необходимо вернуть одномерный вектор'  # Проверяем, что градиент одномерный
        assert grad.shape[0] == len(theta), 'длина вектора должна быть равной количеству весов'  # Проверяем размерность градиента

        # Обновляем параметры модели с помощью градиентного спуска
        theta -= grad * h  # Обновление параметров модели с шагом h

        # Добавляем текущие параметры theta в историю
        theta_history = np.vstack((theta_history, theta))

        # Вычисляем значение логарифмической потери для текущих параметров модели
        loss = logloss(theta, X, y)  # Функция logloss должна быть определена ранее
        errors.append(loss)  # Добавляем ошибку в список

        # Если нужно построить графики
        if plot:
            # Очистка и обновление первого графика (для параметров theta)
            ax1.clear()
            ax1.scatter(range(dim), theta, label='Gradient solution')  # Строим точечный график для theta
            ax1.legend(loc="upper left")  # Добавляем легенду
            ax1.set_title('theta')  # Заголовок графика
            ax1.set_ylabel(r'$\bar \beta$')  # Подпись оси Y
            ax1.set_xlabel('weight ID')  # Подпись оси X

            # Обновляем второй график (для логарифмической потери)
            ax2.plot(range(_+1), errors, 'g-')  # Строим график ошибки (logloss)
            ax2.set_title('logloss')  # Заголовок графика
            ax2.set_xlabel('iterations')  # Подпись оси X

            # Обновляем третий график (для истории изменения параметров theta)
            ax3.plot(theta_history)  # Строим график изменения параметров theta на каждой итерации
            ax3.set_title('update theta')  # Заголовок графика
            ax3.set_ylabel('value')  # Подпись оси Y
            ax3.set_xlabel('iterations')  # Подпись оси X

            # Даем время для обновления графиков
            time.sleep(0.05)  # Задержка между итерациями для визуализации
            fig.canvas.draw()  # Обновляем графики

    # Возвращаем финальные параметры theta после завершения всех итераций
    return theta


In [24]:
# Генерация данных для бинарной классификации с 2000 примерами
X, y = make_classification(n_samples=2000)  
# X — это матрица признаков с 2000 примерами, каждый из которых имеет случайно сгенерированные признаки.
# y — это целевая переменная (метки классов) для 2000 примеров. В бинарной классификации это значения 0 или 1.

In [25]:
optimal_theta = fit(X, y)  
# Обучаем модель с использованием градиентного спуска. Функция fit() будет обновлять параметры модели theta
# на основе обучающих данных (X и y) и возвращать оптимальные параметры theta, которые минимизируют ошибку.


<IPython.core.display.Javascript object>

In [26]:
y_pred = binary_class_prediction(optimal_theta, X)  
# Используем оптимальные параметры theta для предсказания классов на обучающих данных X.
# Функция binary_class_prediction вычисляет вероятности и преобразует их в бинарные метки (0 или 1),
# сравнивая с порогом 0.5.
